<a href="https://colab.research.google.com/github/AzulBarr/Aprendizaje-Automatico/blob/main/TPs/tp2/tp2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install ebooklib beautifulsoup4 pandas

In [ ]:
from ebooklib import epub
import ebooklib
from bs4 import BeautifulSoup
import pandas as pd
import re
from transformers import BertTokenizer, BertModel
import torch
import numpy as np
from torch.utils.data import TensorDataset, DataLoader

## Funciones Auxiliares

In [ ]:
model_name = "bert-base-multilingual-cased"
tokenizer = BertTokenizer.from_pretrained(model_name)
model = BertModel.from_pretrained(model_name)
model.eval()

In [ ]:
def normalize_text_X(t):
    # Convertir a minúsculas y quitar puntuación
    t = t.lower()
    t = re.sub(r'[\u200b-\u200f\uFEFF]', '', t)
    t = re.sub(r"[^a-záéíóúüñ0-9' -]+", ' ', t)
    t = re.sub(r'\s+', ' ', t).strip()
    return t

In [ ]:
def normalize_text_y(t):
    # Convertir a minúsculas y quitar puntuación
    t = re.sub(r'[\u200b-\u200f\uFEFF]', '', t)
    t = re.sub(r"[^a-zA-Záéíóúüñ0-9¿?,.' -]+", ' ', t)
    t = re.sub(r'\s+', ' ', t).strip()
    return t

In [ ]:
def transformar_etiqueta(y, indice):
  puntuacion_iniciales = []
  puntuacion_finales = []
  capitalizaciones = []
  instancia_ids = []
  token_ids = []
  tokens_l = []

  for parrafo in y:
    inicio_pregunta = False
    palabras = parrafo.split()


    for palabra in palabras:
      tokens = tokenizer.tokenize(palabra.lower())

      for i in range(len(tokens)):
        if tokens[i] == "¿":
          inicio_pregunta = True
          continue
        if tokens[i] == "?" or tokens[i] == "." or tokens[i] == ",":
          continue

        instancia_ids.append(indice)
        token_ids.append(tokenizer.convert_tokens_to_ids(tokens[i]))
        tokens_l.append(tokens[i])

        if inicio_pregunta:
          puntuacion_iniciales.append(1) #('"¿"')
          inicio_pregunta = False
        else:
          puntuacion_iniciales.append(0) #("")

        if i != len(tokens) - 1:

          if tokens[i+1] == "?":
            puntuacion_finales.append(3) #('"?"')
          elif tokens[i+1] == ".":
            puntuacion_finales.append(1) #('"."')
          elif tokens[i+1] == ",":
            puntuacion_finales.append(2) #('","')
          else:
            puntuacion_finales.append(0) #("")

        else:
          puntuacion_finales.append(0) #("")

        if palabra.islower():
          capitalizaciones.append(0)
          ultimo_numero = 0
        elif palabra.istitle():
          capitalizaciones.append(1)
          ultimo_numero = 1
        elif palabra.isupper():
          capitalizaciones.append(3)
          ultimo_numero = 3
        else:
          capitalizaciones.append(2)
          ultimo_numero = 2

  etiquetas = np.column_stack([instancia_ids, token_ids, tokens_l, puntuacion_iniciales, puntuacion_finales, capitalizaciones])
  return etiquetas

In [ ]:
def convertir_epub_a_csv(archivo_epub='libro.epub'):
  # Cargar el libro
  book = ebooklib.epub.read_epub(archivo_epub)

  # Lista donde se guardarán los párrafos
  parrafos = []

  # Recorremos los ítems del libro
  for item in book.get_items():
      if item.get_type() == ebooklib.ITEM_DOCUMENT:
          # Parseamos el contenido HTML
          soup = BeautifulSoup(item.get_body_content(), 'html.parser')
          # Extraemos los párrafos
          for p in soup.find_all('p'):
            #print("p:",p, 'tipo: ', type(p))
            texto = p.get_text().strip()
            #print("TEXTO:",texto, ' tipo: ', type(texto))
            palabras = texto.split()
            #print("PALABRAS:",palabras, ' tipo: ', type(palabras))
            if len(palabras) < 20 or len(palabras) > 100:  # descartamos párrafos cortos
                continue
            if texto:
                parrafos.append(texto)

  df = pd.DataFrame({'parrafo': parrafos})
  df.to_csv("libro_parrafos.csv", index=False, encoding="utf-8")

  print(f"Se extrajeron {len(parrafos)} párrafos y se guardaron en 'libro_parrafos.csv'.")

In [ ]:
def crearDataSet(libro):
  convertir_epub_a_csv(libro)
  df = pd.read_csv('libro_parrafos.csv')
  parrafos = pd.DataFrame(columns=['default', 'limpio'])
  parrafos['limpio'] = df['parrafo'].apply(normalize_text_X)
  parrafos['default'] = df['parrafo'].apply(normalize_text_y)

  columnas = ['instancia_id', 'token_id', 'token', 'punt_inicial', 'punt_final', 'capitalización']
  datos = pd.DataFrame(columns=columnas)

  i = 0
  for p in parrafos['default']:
    etiquetas = transformar_etiqueta([p], i)
    etiquetas = pd.DataFrame(etiquetas, columns=columnas)
    datos = pd.concat([datos, etiquetas], ignore_index=True)
    i += 1

  print('Data set creado de tamaño: ', datos.shape)

  numeric_cols = ['instancia_id', 'token_id', 'punt_inicial', 'punt_final', 'capitalización']
  for col in numeric_cols:
    datos[col] = pd.to_numeric(datos[col], errors='coerce')

  return parrafos, datos

In [ ]:
def agregar_embeddings(df_X):
  token_ids = df_X['token_id'].tolist()
  embeddings_list = []
  for token_id  in token_ids:
      if token_id is None or token_id == tokenizer.unk_token_id:
        token_id = tokenizer.unk_token_id
      # Detach the tensor before converting to list
      embedding = model.embeddings.word_embeddings.weight[token_id].detach().tolist()
      embeddings_list.append(embedding)
  df_X['embeddings'] = embeddings_list
  # The embeddings are already lists of floats, no need to convert to int
  # df_X['embeddings'] = df_X['embeddings'].apply(lambda x: [int(i) for i in x])

  return df_X

In [ ]:
def trasformar_df_dfPyTorch(datos_X, datos_Y):
  # Convert the list of lists in the 'embeddings' column to a NumPy array of floats
  embeddings_array = np.array(datos_X['embeddings'].tolist(), dtype=np.float32)
  X = torch.tensor(embeddings_array, dtype=torch.float32)
  Y = torch.tensor(datos_Y[['punt_inicial', 'punt_final', 'capitalización']].values, dtype=torch.float32)
  dataSetPT = TensorDataset(X, Y)

  return dataSetPT

## Cargar libros

Busca libro en formato epub de mi github

In [ ]:
path = 'https://raw.githubusercontent.com/AzulBarr/Aprendizaje-Automatico/main/TPs/tp2'

In [ ]:
libro1 = '/Harry_Potter_y_el_caliz_de_fuego_J_K_Rowling.epub'

In [ ]:
path = path + libro1

In [ ]:
!wget -O libro1.epub $path

##Codigo

In [ ]:
parrafos, dataSet = crearDataSet('libro1.epub')

In [ ]:
datos_X = dataSet[['instancia_id', 'token_id', 'token']]
datos_Y = dataSet[['punt_inicial', 'punt_final', 'capitalización']]

In [ ]:
datos_X_ext = agregar_embeddings(datos_X)

### Puntuacion inicial:
*   "": 0
*   "¿": 1

### Puntuacion final:
*   "": 0
*   ".": 1
*   ",": 2
*   "?": 3



In [ ]:
dataSetPT = trasformar_df_dfPyTorch(datos_X, datos_Y)

In [ ]:
X0, Y0 = dataSetPT[0]
print("X[0]:", X0)
print("Y[0]:", Y0)